<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/v6_calisiyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import copy
import numpy as np
import random
import math

moves=[[-1,0], [-1,-1], [0, -1], [1,-1], [1,0], [1, 1], [0, 1], [-1, 1]]
directions = [1,2,3,4,5,6,7,8];

def find_cost(old_move, new_move):
  if abs(old_move - new_move) == 1 or abs(old_move - new_move) == 7:
    return 1
  elif abs(old_move - new_move) == 2 or abs(old_move - new_move) == 6:
    return 2
  elif abs(old_move - new_move) == 3 or abs(old_move - new_move) == 5:
   return 3
  elif abs(old_move - new_move) == 0:
    return 0
  else:
    return 4

def find_way(end):
  if end[0] == 0 and end[1] == 0:
    return random.randint(5,7)
  elif end[0] == 8 and end[1] == 8:
    return random.randint(1,3)
  elif end[0] == 8 and end[1] == 0:
    return random.choice([1,7,8])
  elif end[0] == 0 and end[1] == 8:
    return random.randint(3,5)
  elif end[0] == 8:
    return random.choice([3,2,1,7,8])
  elif end[0] == 0:
    return random.choice([3,4,5,6,7])
  elif end[1] == 8:
    return random.choice([1,2,3,4,5])
  elif end[1] == 0:
    return random.choice([5,6,7,8,1])
  else:
    return random.randint(1,8)

def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  route_matrix_list = []
  end=[0,0]
  for route in coming_routes:  #population 5000
    flag=0
    cost = 0            
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end[0]=start[0]
    end[1]=start[1]
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:  
          end = [end[0] - add_move[0], end[1] - add_move[1]]
          route[i]=find_way(end)
          add_move =moves[route[i]-1]
          end = [end[0] + add_move[0], end[1] + add_move[1]]
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1  
    route_matrix_list.append(matrix)
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    
    fitness_list.append(find_fitness(start, end, cost, area))
  return return_routes,fitness_list,route_matrix_list

def group(routes,numberOfDrone,fitness_list, matrix_list):
    i=0
    fitness=[]
    routes1_list=[]
    routes2_list=[]
    print("routes len",((len(routes))/2))
    while i<(len(routes)/2):
        sum=similarity(matrix_list[i],matrix_list[i+5000])
        #print("cikti",i+1)
        fitness.append(find_fitness2(fitness_list[i], fitness_list[i+5000], sum))
        routes1_list.append(routes[i])
        routes2_list.append(routes[i+5000])
        #print(sum)
        i=i+1
        
    zipped = list(zip(routes1_list, routes2_list,fitness))   
    sorted_routes = sorted(zipped, key = lambda x: x[2])
    unzipped_list = list(zip(*sorted_routes))
    return unzipped_list

def create_first_route(start,numberOfDrone,length):
  fitness_list = []
  routes = []
  matrix_list = []
  
  for i in range (population*numberOfDrone):
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1;
    end = copy.deepcopy(start)
    route = []
    cost = 0
    route.append(1)
    while len(route) <= length:
      route.append(find_way(end))
      cost += find_cost(route[-2], route[-1])
      add_move = moves[route[-1]-1]
      end[0] += add_move[0]
      end[1] += add_move[1]
      matrix[end[0], end[1]] = 1
    
    area = len(np.argwhere(matrix == 1))
    routes.append(route)
    matrix_list.append(matrix)
    fitness_list.append(find_fitness(start, end, cost, area))
    
  return routes, fitness_list,matrix_list


def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = (area) - cost - (distance)
  return fitness

def cross_over(routes,length):
  new_generation = []
  length=(length/2)+1
  count=0
  for i in range (len(routes)):

    route1 = random.choice(routes)
    route2 = random.choice(routes)
    #new_route1 = route1[:1]+route1[1:21] + route2[21:41] + route1[41:61] + route2[61:]
    #new_route2 = route2[:1]+route2[1:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_route1 = route1[:1]+route1[1:int(length)] + route2[int(length):] 
    new_route2 = route2[:1]+route2[1:int(length)] + route1[int(length):]      

    new_generation.append(new_route1)
    new_generation.append(new_route2)

  return new_generation

def mutation(routes, mutation_ratio, length, population):
  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]        #route sayısı çok azaldığı için hata alıyoruz
    for i in range (3):
      index2 = random.randint(1,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes

def similarity(mat1,mat2):
  sum = 0
  for row1,row2 in zip(mat1,mat2):
    for elem1,elem2 in zip(row1,row2):
      if (elem1 == elem2) & (elem2 == 1):
        sum += 1
  return sum

def find_fitness2(fitness1, fitness2, sum):
  return fitness1 + fitness2 - 8*sum


size = 9
population = 5000
numberOfDrone=2
selected = int(population/2)
generation = 200
length = ((size*size-1)/numberOfDrone)
mutation_ratio = 0.01
matrix_list=[]
matrix_list2=[]
start = [5,5]
end = copy.deepcopy(start)
routes, fitness_list ,matrix_list = create_first_route(start,numberOfDrone,length)

if numberOfDrone >1:
  unzipped_list=group(routes,numberOfDrone,fitness_list,matrix_list)
  print("dslsdkdskl", len(unzipped_list[0]))

  for i in range (generation-1):
      print("i",i)
      new_route_list=[]
      new_fitness_list=[]
      new_matrix_list=[]
      for j in range(numberOfDrone) :
          print("unzipped", len(unzipped_list[j]))
          print("selected",selected)
          cross_generation = cross_over((unzipped_list[j])[selected:], length)
          mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population) 
          new_routes, new_fitness, new_matrix = try_routes(mutated_generation, start, size)
          #print(new_routes)
          new_route_list += new_routes
          #print(new_route_list[0])
          new_fitness_list += new_fitness
          new_matrix_list += new_matrix     
      print("len", len(new_route_list), len(new_fitness_list), len(new_matrix_list))
  unzipped_list=group(new_route_list,numberOfDrone,new_fitness_list,new_matrix_list)
  print(i)
else:
  zipped = list(zip(routes, fitness_list))
  sorted_routes = sorted(zipped, key = lambda x: x[1])
  unzipped_list = list(zip(*sorted_routes))
  new_generation = unzipped_list[0]

  for i in range (generation-1):
    cross_generation = cross_over(new_generation[selected:],length)   
    #print("new gen:",len(cross_generation))
    mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population)
    #print("muntanted gen:",len(mutated_generation))  
    new_routes, new_fitness, matrix_list= try_routes(mutated_generation, start, size)
    #print("new route_list length:", len(new_routes))
    zipped = list(zip(new_routes, new_fitness))
    sorted_routes = sorted(zipped, key = lambda x: x[1])
    unzipped_list = list(zip(*sorted_routes))
    new_generation = unzipped_list[0]
    print("döngü",i+1)
print("bitti")

routes len 5000.0
dslsdkdskl 5000
i 0
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 1
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 2
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 3
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 4
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 5
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 6
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 7
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 8
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 9
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 10
unzipped 5000
selected 2500
unzipped 5000
selected 2500
len 10000 10000 10000
i 11
unzipped 5000
selected 2500
unzipped 5000
selected 2500
le

In [18]:
print((unzipped_list[0][:10]))
print((unzipped_list[1][:10]))
#print(str(unzipped_list[2][5]))
#print(len(unzipped_list[1]))

([1, 5, 4, 3, 7, 6, 2, 5, 2, 8, 5, 4, 1, 8, 3, 1, 4, 4, 5, 2, 8, 2, 5, 7, 4, 7, 7, 8, 8, 4, 1, 3, 3, 7, 5, 1, 7, 3, 1, 5, 7], [1, 4, 3, 5, 2, 5, 1, 2, 8, 4, 2, 1, 6, 1, 7, 8, 1, 5, 1, 3, 2, 5, 8, 3, 5, 2, 5, 5, 7, 2, 7, 3, 5, 1, 5, 6, 3, 6, 1, 7, 1], [1, 3, 5, 5, 7, 5, 3, 1, 3, 4, 2, 6, 3, 7, 8, 1, 3, 7, 1, 4, 1, 4, 6, 4, 7, 2, 5, 7, 7, 1, 5, 1, 2, 2, 5, 3, 8, 6, 7, 1, 5], [1, 3, 4, 4, 5, 7, 2, 4, 7, 1, 3, 8, 3, 1, 1, 4, 5, 1, 5, 5, 6, 8, 8, 4, 7, 1, 5, 3, 2, 1, 5, 2, 6, 8, 4, 1, 1, 7, 5, 7, 2], [1, 2, 5, 4, 8, 4, 1, 8, 2, 3, 5, 6, 1, 2, 8, 2, 3, 6, 1, 5, 1, 2, 5, 8, 5, 3, 2, 5, 7, 2, 5, 8, 4, 8, 5, 8, 5, 3, 5, 8, 2], [1, 2, 1, 4, 8, 5, 7, 2, 3, 5, 2, 4, 4, 7, 7, 2, 7, 8, 2, 2, 8, 5, 6, 2, 4, 1, 3, 5, 1, 1, 5, 1, 5, 1, 7, 7, 3, 6, 5, 8, 7], [1, 5, 3, 1, 3, 7, 8, 2, 3, 3, 5, 7, 5, 8, 3, 3, 4, 5, 4, 7, 6, 1, 7, 4, 3, 1, 6, 3, 2, 6, 3, 1, 5, 1, 1, 5, 7, 1, 7, 5, 3], [1, 7, 2, 3, 8, 8, 6, 4, 8, 2, 8, 8, 4, 8, 3, 6, 1, 4, 6, 4, 7, 4, 2, 1, 2, 3, 5, 5, 1, 7, 3, 4, 2, 7, 7, 4, 3, 5, 5, 8, 8],

In [19]:
end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix1 = np.zeros((size,size))
for i in range (int(length)):
  add_move =moves[unzipped_list[0][0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix1[end[0], end[1]] = 1
print(end)
area1 = len(np.argwhere(matrix1 == 1))
print(area1)

end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix2 = np.zeros((size,size))
for i in range (int(length)):
  add_move =moves[unzipped_list[1][0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix2[end[0], end[1]] = 1
print(end)
area2 = len(np.argwhere(matrix2 == 1))
print(area2)
similarity(matrix1,matrix2)


start: [5, 5]
end [5, 5]
end: [6, 5]
end: [7, 4]
end: [7, 3]
end: [7, 4]
end: [8, 5]
end: [7, 4]
end: [8, 4]
end: [7, 3]
end: [6, 4]
end: [7, 4]
end: [8, 3]
end: [7, 3]
end: [6, 4]
end: [6, 3]
end: [5, 3]
end: [6, 2]
end: [7, 1]
end: [8, 1]
end: [7, 0]
end: [6, 1]
end: [5, 0]
end: [6, 0]
end: [6, 1]
end: [7, 0]
end: [7, 1]
end: [7, 2]
end: [6, 3]
end: [5, 4]
end: [6, 3]
end: [5, 3]
end: [5, 2]
end: [5, 1]
end: [5, 2]
end: [6, 2]
end: [5, 2]
end: [5, 3]
end: [5, 2]
end: [4, 2]
end: [5, 2]
end: [5, 3]
[5, 3]
21
start: [5, 5]
end [5, 5]
end: [4, 4]
end: [5, 3]
end: [5, 4]
end: [5, 3]
end: [6, 4]
end: [6, 3]
end: [7, 3]
end: [6, 2]
end: [7, 1]
end: [8, 0]
end: [7, 0]
end: [6, 1]
end: [7, 1]
end: [6, 1]
end: [7, 0]
end: [6, 1]
end: [5, 2]
end: [4, 2]
end: [5, 1]
end: [6, 1]
end: [5, 2]
end: [6, 1]
end: [5, 1]
end: [5, 0]
end: [6, 1]
end: [6, 2]
end: [7, 1]
end: [6, 2]
end: [7, 3]
end: [8, 4]
end: [7, 3]
end: [8, 2]
end: [8, 3]
end: [7, 2]
end: [8, 1]
end: [8, 0]
end: [7, 0]
end: [6, 0]
end:

18